<a href="https://colab.research.google.com/github/secutron/Practice_Ignite/blob/main/2_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##2.1.구글 Colab 기반 

이그나이트는 작성 시점을 기준으로 다음의 백엔드들을 지원한다.

- backends from native torch distributed configuration: “nccl”, “gloo”, “mpi”
- XLA on TPUs via pytorch/xla
- using Horovod framework as a backend

각 의미를 이해하기 위해 가능한 단순한 내용의 코드를 준비하고, 이를 구글 Colab 기반으로 동작시켜 본다.


### 2.1.1. CPU단 분산처리

  구글 Colab은, 별도로 런타임 유형을 지정하지 않는 경우 GPU나 TPU가 없는 VM(Virtual Machine)이 기본 할당된다. 이는 Colab 페이지의 상단메뉴에서 ‘런타임’ > ‘런타임 유형 변경’ 선택 시 나오는 대화상자가, 아래 그림에서처럼 하드웨어 가속기가 None으로 표시되는 상태인 것으로 확인 가능하다.

 이 상태에서 기본 CPU만을 이용하여 분산처리를 진행해보자.  

<div align="center">
<img width=512 src="https://i.imgur.com/WLjhi9E.png"/>
</div>



####2.1.1.1. 패키지 설치

 먼저 현 시점 기준 Colab에서 제공하는 VM은, 이그나이트가 사전 설치되어 있지 않은 상태이다. 따라서 다음과 같이 이그나이트의 최신 version을 설치한다. 참고로 pip 명령어는 package installer for python)의 약자이며, 아래 명령문 실행 시 이그나이트의 pre-release를 PyPI(python package index)로부터 설치하게 된다.

In [6]:
!pip install --pre pytorch-ignite

결과에 따르면, 본 가이드라인 작성 시점의 최신 버전인 xxxx년 x월 xx일자 (예: 2021년 9월 10일자) 이그나이트 x.x.x (예: 0.5.0)가 설치 되었음을 알 수 있다..

#### 2.1.1.2. 노드 1개, 노드 당 프로세스 수 1개 실행

 설치가 완료되었다면, 아래의 코드를 실행한다. 코드에서 수행하는 작업은 아래와 같다.

~~코드라인 22번부터 31번까지인 idist.Parallel 컨텍스트 매니징 방식만 주의해서 보자. 그 이외는 중요하지 않다~~

 (라인 1)에서는 이그나이트의 distributed 패키지를 idist라는 이름으로 불러들이고, (라인 26-28)에서는 백엔드로 gloo를 할당하는 등의 설정 작업을 지정한 후, (라인 22)에서의 training 함수를, (라인 30-31)에서 컨택스트 매니징이 가능한 idist.Parallel을 이용하여 run 시킨다.

 (라인 22)의 training 함수는 몇 가지 정보를 출력한 후, 무의미한 작업을 반복하도록 작성되었다.

 그리고 (라인 27)에서는 분산처리 설정에 해당하는 dist_configs 딕셔너리에 nproc_per_node 키에 해당하는 값으로 2를 설정하였다. 이는 2개의 자식 프로세스를 생성(spawn)하여 분산 처리를 진행하라는 명령으로 생각하면 된다.


In [7]:
import ignite.distributed as idist

from functools import wraps
import time
import random

def fn_timer(function):
    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.time()
        result = function(*args, **kwargs)
        t1 = time.time()
        print (idist.get_rank(), " : Total time running %s: %s seconds" 
               % (function.__name__, str(t1-t0)))
        return result
    return function_timer

@fn_timer
def random_sort(n):
    return sorted([random.random() for i in range(n)])

def training(local_rank, config, **kwargs):
    print(idist.get_rank(), ': run with config:', config, '- backend=', idist.backend())
    random_sort(2500000)

backend = 'gloo' # or "xla-tpu" or None
dist_configs = {'nproc_per_node': 1, "start_method": "fork"}  # or dist_configs = {...}
config = {'c': 12345}

with idist.Parallel(backend=backend, **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=2)

2021-09-11 01:40:26,746 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'gloo'
2021-09-11 01:40:26,748 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 1
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-09-11 01:40:26,750 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7f39881228c0>' in 1 processes


0 : run with config: {'c': 12345} - backend= gloo
0  : Total time running random_sort: 1.4569926261901855 seconds


2021-09-11 01:40:28,397 ignite.distributed.launcher.Parallel INFO: End of run


결과의 첫번 째 출력 항목은 아래와 같으며, distributed launcher가 gloo 백엔드로 초기화 되었음을 표시한다.  
~~누군가의 실행에 따라 날짜 등은 계속 변한다. 그리고 이그나이트 버전 변경에 따라 표시되는 내용도 변할 수 있다.~~
  
>*2021-06-25 07:48:33,646 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'gloo'*

 상기 ‘’’1.4 분산 딥러닝 기본 지식’’’ 항목에서 언급된 바와 같이 분산처리를 위해서는 컴퓨팅 코어 간 통신이 필요하며, 본 예제에서는 컴퓨팅 코어가 CPU인 경우이므로 gloo나 mpi이 백엔드로 이용된 것이다. 그리고 만일 이 컴퓨팅 코어가 GPU인 경우 nccl 백엔드 이용이 가능하다. 

 이와 관련해서는 DISTRIBUTED COMMUNICATION PACKAGE - TORCH.DISTRIBUTED 페이지의 [rule of thumb](https://pytorch.org/docs/stable/distributed.html) 항목을 참조한다. ~~분산처리의 역사만큼이나 다양한 분산처리 방식이 존재한다. TCP 등을 이용해 직접 프로세스간 통신을 처리하는 방법도 있지만, 하이레벨 관점에서 CPU는 gloo, GPU는 nccl, TPU는 xla 백엔드를 사용해야 한다고 생각하는 것이 정신건강에 좋다.~~

  두번 째 출력 항목은 node 1개(nnodes: 1)에서, 노드 당 1개의 분산처리 작업 (nproc_per_node: 1)이 진행될 것임을 알려준다. 


>*2021-06-25 07:48:33,649 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes:*  
>>*nproc_per_node: 1*  
	*nnodes: 1*  
	*node_rank: 0*  
	*start_method: fork*  

 세번 째 출력 항목은 각 프로세스 내에서 print문에 의한 출력과 측정된 소요 시간을 보여주고 있다. rank가 0번 하나로 표기되었고, 이는 프로세스 총 합이 1개임을 보여 준다.

>*2021-06-25 07:48:33,651 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7ff2ddc0b440>' in 1 processes*
>>*0 : run with config: {'c': 12345} - backend= gloo*  
*0  : Total time running random_sort: 2.119020938873291 seconds*



 그리고 마지막 출력 항목은 분산처리가 모두 끝났음을 알리고 있다.

>*2021-06-25 07:48:36,009 ignite.distributed.launcher.Parallel INFO: End of run*


 여기서 프로세스 시작을 알리는 시간(세번 째 출력 항목)이 07시 48분 33초이고, 분산처리가 모두 끝났음을 알리는 시간(마지막 출력 항목)은 07시 48분 36초임을 확인해 보자. 각 2.1초 정도의 시간이 소요되는 작업(training 함수)을 총 1번 진행하였으며, 전체 처리 시간은 약 2.5초 소요되었음을 알려주고있다.   
 ~~처리 시간은 Colab에서 할당해주는 VM 종류에 따라 변할 수 있다~~
 

#### 2.1.1.3. 노드 1개, 노드 당 프로세스 수 2개 실행

 만일 노드 당 분산처리 작업(nproc_per_node)을 2개로 바꾸면 어떻게 될까? 이를 위해 아래의 코드를 추가하고 실행한다. 코드에서는 노드 당 2개의 분산처리 작업을 수행할 수 있도록 nproc_per_node 항목에 2를 할당하였다.


In [8]:
dist_configs['nproc_per_node'] = 2

with idist.Parallel(backend=backend, **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=2)

2021-09-11 01:40:28,412 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'gloo'
2021-09-11 01:40:28,414 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 2
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-09-11 01:40:28,417 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7f39881228c0>' in 2 processes


0 : run with config: {'c': 12345} - backend= gloo
1 : run with config: {'c': 12345} - backend= gloo
1  : Total time running random_sort: 2.355760097503662 seconds
0  : Total time running random_sort: 2.5288949012756348 seconds


2021-09-11 01:40:31,209 ignite.distributed.launcher.Parallel INFO: End of run


아래 출력 항목은 node 1개(nnodes: 1)에서, 노드 당 2개의 분산처리 작업 (nproc_per_node: 2)이 진행될 것임을 알려준다. 

>*2021-06-25 05:48:33,629 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes:*  
	>>*nproc_per_node: 2*  
	*nnodes: 1*  
	*node_rank: 0*  
	*start_method: fork*  



 그리고 아래 출력 항목은 각 프로세스 내에서 print문에 의한 출력과 측정된 소요 시간을 보여주고 있다. rank가 각 0번과 1번으로 표기되었고, 프로세스 총 합은 2개임을 보여 준다.

>*2021-06-25 05:48:33,651 이그나이트.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7ff2ddc0b440>' in 2 processes*  
*1 : run with config: {'c': 12345} - backend= gloo*  
*0 : run with config: {'c': 12345} - backend= gloo*  
*0  : Total time running random_sort: 2.119020938873291 seconds*  
*1  : Total time running random_sort: 2.177091121673584 seconds*  


 그리고 마지막 출력 항목은 분산처리가 모두 끝났음을 알리고 있다.

>*2021-06-25 05:48:36,619 이그나이트.distributed.launcher.Parallel INFO: End of run*


 여기서 프로세스 시작을 알리는 시간(세번 째 출력 항목)이 05시 48분 33초이고, 분산처리가 모두 끝났음을 알리는 시간(마지막 출력 항목)은 05시 48분 36초임을 확인해 보자. 각 2.1초 정도의 시간이 소요되는 작업(training 함수)을 총 2번 진행하였으나, 분산처리의 도움으로 전체 처리 시간은 3초에 그친다. 


#### 2.1.1.4. 노드 1개, 노드 당 프로세스 수 8개 실행

 만일 노드 당 분산처리 작업(nproc_per_node)를 8개로 바꾸면 어떻게 될까?


In [9]:
dist_configs['nproc_per_node'] = 8

with idist.Parallel(backend=backend, **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=2)

2021-09-11 01:40:31,223 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'gloo'
2021-09-11 01:40:31,226 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 8
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-09-11 01:40:31,228 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7f39881228c0>' in 8 processes


7 : run with config: {'c': 12345} - backend= gloo
0 : run with config: {'c': 12345} - backend= gloo
1 : run with config: {'c': 12345} - backend= gloo
2 : run with config: {'c': 12345} - backend= gloo
6 : run with config: {'c': 12345} - backend= gloo
3 : run with config: {'c': 12345} - backend= gloo
5 : run with config: {'c': 12345} - backend= gloo
4 : run with config: {'c': 12345} - backend= gloo
2  : Total time running random_sort: 9.545878648757935 seconds
7  : Total time running random_sort: 9.645442962646484 seconds
6  : Total time running random_sort: 9.782885313034058 seconds
3  : Total time running random_sort: 9.876355648040771 seconds
5  : Total time running random_sort: 9.961318731307983 seconds
1  : Total time running random_sort: 10.032445430755615 seconds
4  : Total time running random_sort: 10.000553131103516 seconds
0  : Total time running random_sort: 10.293447017669678 seconds


2021-09-11 01:40:42,497 ignite.distributed.launcher.Parallel INFO: End of run


다음과 같이 8개의 child process가 생성되어 작업이 진행되었음을 알 수 있다.

>*2021-06-25 07:44:08,255 이그나이트.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'gloo'  
2021-06-25 07:44:08,256 이그나이트.distributed.launcher.Parallel INFO: - Parameters to spawn processes:*
	>>*nproc_per_node: 8*  
	*nnodes: 1*  
	*node_rank: 0*  
	*start_method: fork*  

>*2021-06-25 07:44:08,259 이그나이트.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7ff2ddc0b560>' in 8 processes*  
*0 : run with config: {'c': 12345} - backend= gloo*  
*4 : run with config: {'c': 12345} - backend= gloo*  
*5 : run with config: {'c': 12345} - backend= gloo*  
*3 : run with config: {'c': 12345} - backend= gloo*  
*2 : run with config: {'c': 12345} - backend= gloo*  
*1 : run with config: {'c': 12345} - backend= gloo*  
*6 : run with config: {'c': 12345} - backend= gloo*  
*7 : run with config: {'c': 12345} - backend= gloo*  
*7  : Total time running random_sort: 8.505528688430786 seconds*  
*0  : Total time running random_sort: 8.733642339706421 seconds*  
*6  : Total time running random_sort: 8.731879472732544 seconds*  
*4  : Total time running random_sort: 8.774088859558105 seconds*  
*3  : Total time running random_sort: 8.826892614364624 seconds*  
*2  : Total time running random_sort: 8.832333087921143 seconds*  
*1  : Total time running random_sort: 8.853899955749512 seconds*  
*5  : Total time running random_sort: 8.880859375 seconds*  

>*2021-06-25 07:44:19,250 이그나이트.distributed.launcher.Parallel INFO: End of run*




 07시 44분 08초에 생성되어, 07시 44분 19초에 작업이 끝났음을 눈여겨 보자. 프로세스간 통신 등의 오버헤드로 인해 좀 더 시간이 소요되었으나 여전히 전체 시간은 단축되었음을 알 수 있다. 


물론 아래와 같이 더 많은 수의 프로세스를 생성하여 작업하는 것도 가능하다.

In [10]:
dist_configs['nproc_per_node'] = 50

with idist.Parallel(backend=backend, **dist_configs) as parallel:
    parallel.run(training, config, a=1, b=2)

2021-09-11 01:40:42,509 ignite.distributed.launcher.Parallel INFO: Initialized distributed launcher with backend: 'gloo'
2021-09-11 01:40:42,512 ignite.distributed.launcher.Parallel INFO: - Parameters to spawn processes: 
	nproc_per_node: 50
	nnodes: 1
	node_rank: 0
	start_method: fork
2021-09-11 01:40:42,514 ignite.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7f39881228c0>' in 50 processes


30 : run with config: {'c': 12345} - backend= gloo
45 : run with config: {'c': 12345} - backend= gloo
32 : run with config: {'c': 12345} - backend= gloo
40 : run with config: {'c': 12345} - backend= gloo
9 : run with config: {'c': 12345} - backend= gloo
42 : run with config: {'c': 12345} - backend= gloo
43 : run with config: {'c': 12345} - backend= gloo
16 : run with config: {'c': 12345} - backend= gloo
23 : run with config: {'c': 12345} - backend= gloo
47 : run with config: {'c': 12345} - backend= gloo
3 : run with config: {'c': 12345} - backend= gloo
6 : run with config: {'c': 12345} - backend= gloo
13 : run with config: {'c': 12345} - backend= gloo
18 : run with config: {'c': 12345} - backend= gloo
36 : run with config: {'c': 12345} - backend= gloo
27 : run with config: {'c': 12345} - backend= gloo
10 : run with config: {'c': 12345} - backend= gloo
44 : run with config: {'c': 12345} - backend= gloo
38 : run with config: {'c': 12345} - backend= gloo
28 : run with config: {'c': 12345}

2021-09-11 01:42:08,647 ignite.distributed.launcher.Parallel INFO: End of run


#### 2.1.1.5. VM 리소스 확인

 이제 상기 분산처리 작업을 진행한 Colab의 VM에 대해 확인해보자.


In [11]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 1999.999
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	:

 상기 실험 시 Colab으로부터 할당받은 VM은 총 2개의 인텔 제온 프로세서가 탑재되어 있고, 각 프로세서 당 코어 수는 1개이다. 따라서, 총 2개의 코어에서 8개의 (논리적) child process를 생성하여 분산처리 작업이 진행되었다는 것을 알 수 있다.

>*processor	: 0*  
...  
*model name	: Intel(R) Xeon(R) CPU @ 2.30GHz*  
...  
*cpu cores	: 1*  
...  
*processor	: 1*  
...  
*model name	: Intel(R) Xeon(R) CPU @ 2.30GHz*  
...  
*cpu cores	: 1*  
...  


#### 2.1.1.5. 결과 해석

 이제까지 Colab 기반으로 CPU단에서 분산처리를 진행하는 방식을 살펴보았다. 그리고 ‘’’2.1.1.4절의 VM 리소스 확인’’’ 항목에서, 분산처리를 위해 Colab으로부터 할당받은 VM이 2-CPU로 구성된 것이었음을 확인하였다. 

 >*참고로 2-CPU, 즉 보드 위에 여러 개의 CPU를 탑재하는 multi-CPU 방식은 2000년대 초반 시절에 [무어의 법칙](https://en.wikipedia.org/wiki/Moore%27s_law)/[폴락의 법칙](https://en.wikipedia.org/wiki/Pollack%27s_rule) 극복을 위해 고려되던 것이다. multi-CPU방식은 CPU 별로 어드레싱을 분리할 수 있으므로 사용 가능한 램 용량을 크게 늘릴 수 있다는 등의 장점도 있다. 그러나 이후 하나의 다이(die)에 2개의 코어를 얹는 인텔 코어2가 큰 성공을 거두면서, 멀티코어(multi-core) 방식이 새로운 주류가 되었다. 현 시점 기준으로는 인텔의 최신 10세대 Core i9이 10코어/20쓰레드를 지원하고, AMD는 3세대 Ryzen 9에서 인텔보다 앞서는 16코어/32쓰레드를 지원한다. ~~AMD 직원 대상 리사 수 지지율 조사 결과 무려 98%가 지지!!~~* 

<br/>

 분산처리에서는 여러 개의 프로세스를 생성하여 동시에 작업을 진행시키는 것이 중요하다. 여기서 동시에 작업을 진행시킨다는 의미는 동시에 작업을 하는 것처럼 보이는 것([concurrent computing](https://en.wikipedia.org/wiki/Concurrent_computing))이 아니라, 말 그대로 동시에 작업이 진행되는 것([parallel computing](https://en.wikipedia.org/wiki/Parallel_computing))을 말한다. 

 코어 1개에서는 1개의 프로세스가 실행될 수 있다. 시분할 스케쥴링 등의 방법을 통해 여러 개의 프로세스가 동시에 실행되는 것처럼 보일 수 있지만, 실제로 한 시점에 실행되는 프로세스는 코어당 1개이다. ~~예외로 인텔의 하이퍼쓰레드 같은 기술이 있지만 단순화를 위해 언급하지 않는다.~~  따라서 2-코어의 경우 2개의 프로세스가 동시에 실행될 수 있으며, 이는 Colab으로부터 할당받은 2-CPU의 경우에도 동일하다. ~~Colab의 2-CPU는, 분산처리 관점에서 multi-core의 하나인 dual-core 경우와 동일하게 취급해도 무방하다.~~ 

<br/>

 이를 상기 ‘’’2.1.1.2 노드 1개, 노드 당 프로세스 수 2개 실행” 항목에서 얻은 결과와 비교하여 생각해 보자. 

>*2021-06-25 07:48:33,651 이그나이트.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7ff2ddc0b440>' in 2 processes*  
*1 : run with config: {'c': 12345} - backend= gloo*  
*0 : run with config: {'c': 12345} - backend= gloo*  
*0  : Total time running random_sort: 2.119020938873291 seconds*  
*1  : Total time running random_sort: 2.177091121673584 seconds*  


 개별 프로세스 당 약 2.1초가 소요되는 작업을 진행하였고, 분산처리 기준의 전체 작업시간은 약 3초이다. 그리고 결과 메시지에 포함된 rank id 0번과 1번 표시로부터, 할당받은 VM에 포함된 2개의 CPU에 프로세스가 각각 1개씩 분할되어 작업이 진행되었음을 확인할 수 있다. 또한 전체 작업 시간이 3초로서, 개별 프로세스 당 작업 시간인 2.1초보다 약 0.9초 더 소요된 이유는 데이터의 교환, 컨텍스트 스위치로 인한 캐시 적중 실패 등의 오버헤드로 인한 것이다. 이에 대해 좀 더 자세히 알고 싶은 경우 [암달의 법칙](https://en.wikipedia.org/wiki/Amdahl%27s_law) 링크를 참조한다.

<br/>

 이제 ‘’’2.1.1.3 노드 1개, 노드 당 프로세스 수 8개 실행”항목에서 얻은 결과와도 비교해본다.

>*2021-06-25 07:44:08,259 이그나이트.distributed.launcher.Parallel INFO: Spawn function '<function training at 0x7ff2ddc0b560>' in 8 processes*  
*0 : run with config: {'c': 12345} - backend= gloo*  
*4 : run with config: {'c': 12345} - backend= gloo*  
*5 : run with config: {'c': 12345} - backend= gloo*  
*3 : run with config: {'c': 12345} - backend= gloo*  
*2 : run with config: {'c': 12345} - backend= gloo*  
*1 : run with config: {'c': 12345} - backend= gloo*  
*6 : run with config: {'c': 12345} - backend= gloo*  
*7 : run with config: {'c': 12345} - backend= gloo*  
*7  : Total time running random_sort: 8.505528688430786 seconds*  
*0  : Total time running random_sort: 8.733642339706421 seconds*  
*6  : Total time running random_sort: 8.731879472732544 seconds*  
*4  : Total time running random_sort: 8.774088859558105 seconds*  
*3  : Total time running random_sort: 8.826892614364624 seconds*  
*2  : Total time running random_sort: 8.832333087921143 seconds*  
*1  : Total time running random_sort: 8.853899955749512 seconds*  
*5  : Total time running random_sort: 8.880859375 seconds*  
*2021-06-25 07:44:19,250 이그나이트.distributed.launcher.Parallel INFO: End of run*  

 앞에서는 각 프로세스 당 약 2.1초가 소요되었는데, 이번에는 프로세스 당 8.7초에서 8.8초가 소요되었음을 알 수 있다. 이는 2-CPU에 8개의 분산처리 작업이 요청됨에 따라, 마치 동시에 모든 작업이 처리되는 것처럼 보이도록 진행되었기 때문이다. 

 그럼에도 불구하고, 각 CPU 코어 관점에서 볼 때 여러 개의 프로세스가 교체되는 오버헤드가 추가 발생했지만 2.1초 분량의 작업이 8차례 순차 작업되는 시간보다는 줄어들었음을 알 수 있다.


<br/>

 그리고 아래 그림은 wandb(weight and bias) tool을 이용하여 추적한 CPU utilization 확인 결과이다. 

<div align="center">
<img width=512 src="https://i.imgur.com/7AKuf4F.png"/>
</div>



Note: This is not an official [LG AI Research](https://www.lgresearch.ai/) product but sample code provided for an educational purpose

<br/>
author: John H. Kim
<br/>  
email: john.kim@lgresearch.ai / secutron@naver.com  
